In [1]:
%reload_ext autoreload
%autoreload 2

import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [3]:
from app.system import System
from app.assistant import Assistant
from app.assistant_response_router import AssistantResponseRouter

from app.tools import PropertiesFilterTool, InfoExtractorTool
from app.assistant_response_router import AssistantResponseRouter
from app.agents import load_information_agent, get_create_booking_agent, get_booking_conversation_agent, get_house_selection_agent


In [3]:
system = System()
assistant = Assistant()
assistant_response_router = AssistantResponseRouter()



In [4]:
def handler(message, customer_number="99513718"):
    conversation = system.get_conversation(customer_number)
    print(f"Conversation: {conversation}")
    # create_booking_agent = get_create_booking_agent(conversation)
    create_booking_agent = get_booking_conversation_agent(conversation)
    system.add_user_message(message, customer_number)
    print(f"New Conversation: {system.get_conversation(customer_number)}")

    
    assistant_response = create_booking_agent.run(input=message)
    system.add_assistant_message(assistant_response, customer_number)
    # assistant_response = assistant.run(message, conversation)
    print(f"Assistant Response: {assistant_response}")

    # # assistant_response_router_input = system.get_conversation_string(customer_number)
    # print(f"Assistant Response Router Input: {assistant_response}")
    # next_action = assistant_response_router(assistant_response)
    # print(f"Next Action: {next_action}")
    # if "load_properties_information" in next_action["action"]:
    #     print(f"Loading properties...")
    #     properties_info_response = load_information_agent.run(input=system.get_conversation_string(customer_number))
    #     if properties_info_response != "":
    #         # system.add_assistant_message(properties_info_response, customer_number)
    #         print(f"Updating conversation with Properties loaded")
    #         system.add_system_message(properties_info_response["text"], customer_number)
    #         print(f"Getting new response from Assistant")
    #         new_assistant_response = assistant.get_completion_from_messages(system.get_conversation(customer_number))
    #         system.add_assistant_message(new_assistant_response, customer_number)
    #         return new_assistant_response
    #     else:
    #         print(f"Not information loaded from properties, continuew conversation.")
    #         system.add_assistant_message(assistant_response, customer_number)
    #         return assistant_response
    # elif "pre_booking" in next_action["action"]:
    #     print(f"Creating pre booking...")
    #     properties_info_response = create_booking_agent.run(input=system.get_conversation_string(customer_number))
    #     print(f"Booking created: {properties_info_response}")
    # else:
    #     print(f"Continue with conversation.")
    #     system.add_assistant_message(assistant_response, customer_number)
    #     return assistant_response



In [5]:
#handler("Hola", customer_number="99513718")
handler("Me gustaria reservar una casa para 3 personas por dos noches para el 20 de junio", customer_number="99513718")
#handler("22 de julio", customer_number="99513718")

#handler("Cualquiera me sirve", customer_number="99513718")

#handler("Perfecto, me gustaria reservar la propiedad Sol", customer_number="99513718")
#handler("Agustin Sarasua, a@gmail.com, 099513718", customer_number="99513718")


#handler(" 3 personas para el 20 de junio por 2 noches", customer_number="99513718")
#handler("En mercedes, la mas economica", customer_number="99513718")

#handler("Perfecto, mer sirve", customer_number="99513718")
#handler("Aca te paso: Agustin Sarasua, a@gmail.com, 099513718", customer_number="99513718")



Conversation: []
New Conversation: [{'role': 'user', 'content': 'Me gustaria reservar una casa para 3 personas por dos noches para el 20 de junio'}]


> Entering new  chain...
{
    "action": "info_extractor",
    "action_input": "Me gustaria reservar una casa para 3 personas por dos noches para el 20 de junio"
}

> Entering new  chain...
Prompt after formatting:
Given a query containing information about a booking of an accomodation extract the check-in date, checkout-date and number of nights from the query. If the year is not specified in the query, assume it is after today, same year or at most next year if the month is before current month.
Today is 18 June 2023.

Here is the query:
Me gustaria reservar una casa para 3 personas por dos noches para el 20 de junio

> Finished chain.

Observation: 

Check-in date: 20 June 2023
Checkout date: 22 June 2023
Number of nights: 2
Thought:{
    "action": "Final Answer",
    "action_input": "You can reserve a house for 3 people at Complejo E

In [6]:
handler("La cabaña Sol, por favor", customer_number="99513718")

# info_extractor_tool = InfoExtractorTool()
# info_extractor_tool_result = info_extractor_tool.run({"query": "Hola, me gustaria alquilar una casa para el 2 de setiembre para 3 personas por 2 noches"})
# info_extractor_tool_result
# # models = get_models_tool.run({"query_params": {"search": "gpt-j"}})


# properties_filter_tool = PropertiesFilterTool()
# properties_filter_tool.run({'checkin_date': '2023-06-20',
#  'checkout_date': '2023-06-23',
#  'num_nights': '3',
#  'num_guests': '3'})


Conversation: [{'role': 'user', 'content': 'Me gustaria reservar una casa para 3 personas por dos noches para el 20 de junio'}, {'role': 'assistant', 'content': "Tenemos dos propiedades disponibles para alquilar. La primera es la Cabaña 'Sol', una lujosa villa con vistas panorámicas a las montañas, una piscina privada, jardines exuberantes y una terraza para disfrutar de las maravillosas vistas. Esta propiedad cuenta con Wi-Fi, estacionamiento privado, mascotas permitidas, parrilla y un precio de 250 USD por noche. La segunda propiedad es la Cabaña 'Luna', que ofrece los mismos servicios que la Cabaña 'Sol', pero con un precio de 120 USD por noche. ¿Cuál prefieres?"}]
New Conversation: [{'role': 'user', 'content': 'Me gustaria reservar una casa para 3 personas por dos noches para el 20 de junio'}, {'role': 'assistant', 'content': "Tenemos dos propiedades disponibles para alquilar. La primera es la Cabaña 'Sol', una lujosa villa con vistas panorámicas a las montañas, una piscina privada

In [14]:
new_conv = system.get_conversation("99513718")[:2]
system.save_conversation("99513718", new_conv)
new_conv

[{'role': 'user',
  'content': 'Me gustaria reservar una casa para 3 personas por dos noches para el 20 de junio'},
 {'role': 'assistant',
  'content': "Tenemos dos propiedades disponibles para alquilar. La primera es la Cabaña 'Sol', una lujosa villa con vistas panorámicas a las montañas, una piscina privada, jardines exuberantes y una terraza para disfrutar de las maravillosas vistas. La segunda es la Cabaña 'Luna', que ofrece los mismos servicios que la Cabaña 'Sol'. Ambos alojamientos cuentan con Wi-Fi, estacionamiento privado, mascotas permitidas, barbacoa y piscina privada. ¿Cuál de las dos propiedades le gustaría reservar?"}]

In [12]:
#handler("Si, por favor", customer_number="99513718")
system.get_conversation("99513718")

[{'role': 'user',
  'content': 'Me gustaria reservar una casa para 3 personas por dos noches para el 20 de junio'},
 {'role': 'assistant',
  'content': "Tenemos dos propiedades disponibles para alquilar. La primera es la Cabaña 'Sol', una lujosa villa con vistas panorámicas a las montañas, una piscina privada, jardines exuberantes y una terraza para disfrutar de las maravillosas vistas. La segunda es la Cabaña 'Luna', que ofrece los mismos servicios que la Cabaña 'Sol'. Ambos alojamientos cuentan con Wi-Fi, estacionamiento privado, mascotas permitidas, barbacoa y piscina privada. ¿Cuál de las dos propiedades le gustaría reservar?"}]

In [8]:
handler("Agustin Sarasua, agustin@gmail.com", customer_number="99513718")

Conversation: [{'role': 'user', 'content': 'Me gustaria reservar una casa para 3 personas por dos noches para el 20 de junio'}, {'role': 'user', 'content': 'La cabaña Sol, por favor'}, {'role': 'user', 'content': 'Si, por favor'}]
New Conversation: [{'role': 'user', 'content': 'Me gustaria reservar una casa para 3 personas por dos noches para el 20 de junio'}, {'role': 'user', 'content': 'La cabaña Sol, por favor'}, {'role': 'user', 'content': 'Si, por favor'}, {'role': 'user', 'content': 'Agustin Sarasua, agustin@gmail.com'}]


> Entering new  chain...
Action:
```
{
  "action": "info_extractor",
  "action_input": {
    "query": "Reservar la cabaña Sol para 3 personas por dos noches para el 20 de junio. Agustin Sarasua, agustin@gmail.com"
  }
}
``` 


> Entering new  chain...
Prompt after formatting:
Given a conversation between a user and an assistant about booking a house for short-term stay. Your job is to extract the check-in date, checkout-date and number of nights from the query.

In [1]:

# make_reservation_task = Task(
#     "MAKE_RESERVATION", 
#     [Step("GATHER_BOOKING_INFO"), Step("HOUSE_SELECTION"), Step("GATHER_USER_INFORMATION"), Step("CHECKOUT")])



In [4]:
from app.system import Conversation, Task, Step
from app.task_resolver import GatherBookingInfoResolver
from app.system import System

system_x = System()
step = Step("GATHER_BOOKING_INFO", GatherBookingInfoResolver())
customer_number="99513718"

def test_step(message, customer_number="99513718"):
    system_x.add_user_message(message, customer_number)
    conversation = system_x.get_conversation(customer_number)
    print(f"Conversation: {conversation}")
    result = step.resolve(conversation)

    print(f"Result: {result} \nStep Done: {step.is_done()}")
    if not step.is_done():
        system_x.add_assistant_message(result, customer_number)
    else:
        print("Step is Done, we can move to the next step.")

In [5]:
#test_step("Hola")
test_step("2 personas para el sabado por 2 noches")

Conversation: [{'role': 'user', 'content': '2 personas para el sabado por 2 noches'}]
Assistant Response: ¡Hola! Gracias por contactarnos. ¿Podrías proporcionarme la fecha de check-in y la fecha de check-out por favor?


> Entering new  chain...
Prompt after formatting:
Given a conversation between a user and an assistant about booking a house for short-term stay. Your job is to extract the check-in date, checkout-date and number of nights from the query.
If the year is not specified in the query, assume it is after today, same year or at most next year if the month is before current month.
Today is 19 June 2023.



Here is the conversation: 
user: 2 personas para el sabado por 2 noches


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"checkin_date": string  // reservation Check-In date formated as YYYY-MM-DD i.e: 2023-09-24. If not present, set an empty string.
	"checkout_date": string

In [1]:
from app.system import Conversation, Task, Step
from app.task_resolver import GatherBookingInfoResolver, HouseSelectionResolver, GatherUserInfoResolver, BookingConfirmationResolver
from app.system import System
import langchain

system_x = System()
customer_number="99513718"

langchain.debug = False
gather_booking_info_step = Step("GATHER_BOOKING_INFO", GatherBookingInfoResolver(), reply_when_done=False)
# gather_booking_info_step.data = {
#     "booking_information": {'checkin_date': '2023-06-19', 'checkout_date': '2023-06-21', 'num_nights': 2, 'num_guests': 2}
# }
# gather_booking_info_step.status = "DONE"
house_selection_step = Step("HOUSE_SELECTION", HouseSelectionResolver())

gather_user_info_step = Step("GATHER_USER_INFO", GatherUserInfoResolver(), reply_when_done=False)

booking_confirmation_step = Step("BOOKING_CONFIRMATION", BookingConfirmationResolver())

reservation_task = Task("RESERVATION_TASK", [gather_booking_info_step, house_selection_step, gather_user_info_step, booking_confirmation_step])

def test_task(message, customer_number="99513718"):
    
    system_x.add_user_message(message, customer_number)
    conversation = system_x.get_conversation(customer_number)
    print(f"Conversation: {conversation}")
    response = reservation_task.run(conversation)
    system_x.add_assistant_message(response, customer_number)
    return response
    # system_x.add_user_message(message, customer_number)
    # conversation = system_x.get_conversation(customer_number)
    # print(f"Conversation: {conversation}")
    # result = step.resolve(conversation)

    # print(f"Result: {result} \nStep Done: {step.is_done()}")
    # if not step.is_done():
    #     system_x.add_assistant_message(result, customer_number)
    # else:
    #     print("Step is Done, we can move to the next step.")

In [5]:
# test_task("Hola, me gustaria reservar una casa para el sabado que viene por dos noches para 3 personas.")
# test_task("La mas barata")
# test_task("Agustin Sarasua, agustinsarasua@gmail.com")
test_task("Si")
# test_task("Si, me gustaria reservar la mas barata")

Conversation: [{'role': 'user', 'content': 'Hola, me gustaria reservar una casa para el sabado que viene por dos noches para 3 personas.'}, {'role': 'assistant', 'content': 'Hola! Tenemos dos propiedades disponibles: \n\n1. Cabaña Sol: Impresionante villa con vistas panorámicas a las montañas. Esta lujosa propiedad ofrece un ambiente tranquilo y relajante con amplios espacios interiores y exteriores. Cuenta con una piscina privada, jardines exuberantes y una terraza para disfrutar de las maravillosas vistas. Perfecta para escapadas en familia o con amigos. Precio por noche: 250.0 USD. \n\n2. Cabaña Luna: Impresionante villa con vistas panorámicas a las montañas. Esta lujosa propiedad ofrece un ambiente tranquilo y relajante con amplios espacios interiores y exteriores. Cuenta con una piscina privada, jardines exuberantes y una terraza para disfrutar de las maravillosas vistas. Perfecta para escapadas en familia o con amigos. Precio por noche: 120.0 USD. \n\nPor favor, seleccione una pr

TypeError: list indices must be integers or slices, not str

In [9]:
from app.agents import load_information_agent, get_create_booking_agent, get_booking_conversation_agent, get_house_selection_agent

from app.system import Conversation, Task, Step
from app.task_resolver import GatherBookingInfoResolver
from app.system import System

system_x = System()
# step = Step("GATHER_BOOKING_INFO", GatherBookingInfoResolver())
customer_number="99513718"

def test_agent(message, customer_number="99513718"):
    
    conversation = system_x.get_conversation(customer_number)
    print(f"Conversation: {conversation}")
    house_selection_agent = get_house_selection_agent(conversation)
    system_x.add_user_message(message, customer_number)

    assistant_response = house_selection_agent.run(input=message)
    system_x.add_system_message(assistant_response, customer_number)
    return assistant_response

In [ ]:
booking_information = {'checkin_date': '2023-06-19', 'checkout_date': '2023-06-21', 'num_nights': 2, 'num_guests': 2}

msg = f"""Me gustaria elegir una casa con la siguiente disponibilidad: \
check in: {booking_information['checkin_date']}, \
check out: {booking_information['checkout_date']}, \
numero de personas: {booking_information['num_guests']}, \
numero de noches: {booking_information['num_nights']}"""

test_agent(msg)

In [ ]:
test_agent("Bien, cual es mas barata?")

In [7]:
import json

def format_json(json_data):
    formatted_string = ''
    data = json.loads(json_data)
    for property_name, property_data in data.items():
        formatted_string += f"{property_name}:\n"
        for key, value in property_data.items():
            formatted_string += f"{key}: {value}\n"
        formatted_string += "\n"
    return formatted_string

# Example usage
json_data = '''
{
    "Luna": {
        "property_id": "Luna",
        "description": "Impresionante villa con vistas panorámicas a las montañas.",
        "amenities": ["Wi-Fi", "private parking", "pet-friendly", "barbecue", "private pool"],
        "price": "120",
        "currency": "USD",
        "max_guests": "4"
    }
}
'''

format_json(json_data)


"Luna:\nproperty_id: Luna\ndescription: Impresionante villa con vistas panorámicas a las montañas.\namenities: ['Wi-Fi', 'private parking', 'pet-friendly', 'barbecue', 'private pool']\nprice: 120\ncurrency: USD\nmax_guests: 4\n\n"